In [5]:
import numpy as np
import pandas as pd

In [6]:
pop = pd.read_csv('C:\\Users\\acer\\ml\\datasets\\US State\\state-population.csv')
areas = pd.read_csv('C:\\Users\\acer\\ml\\datasets\\US State\\state-areas.csv')
abbrevs = pd.read_csv('C:\\Users\\acer\\ml\\datasets\\US State\\state-abbrevs.csv')

In [8]:
pop.head()

state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

In [9]:
areas.head()

state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

In [10]:
abbrevs.head()

state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

### we want to compute : rank US states and territories by their 2010 population density

In [20]:
# STEP 1 : perform many to one merge on pop and abbrevation to get the  full name of state

mergedPopAbbre = pd.merge(pop, abbrevs, how='outer', 
                  left_on = 'state/region', right_on = 'abbreviation')
mergedPopAbbre = merged.drop('abbreviation', 1)
mergedPopAbbre.head()

state/region     ages  year  population    state
0           AL  under18  2012   1117489.0  Alabama
1           AL    total  2012   4817528.0  Alabama
2           AL  under18  2010   1130966.0  Alabama
3           AL    total  2010   4785570.0  Alabama
4           AL  under18  2011   1125763.0  Alabama

In [22]:
#Step 2 : check for null values
mergedPopAbbre.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [24]:
#lets figure out which values are none
mergedPopAbbre[mergedPopAbbre['population'].isnull()].head()

state/region     ages  year  population state
2448           PR  under18  1990         NaN   NaN
2449           PR    total  1990         NaN   NaN
2450           PR    total  1991         NaN   NaN
2451           PR  under18  1991         NaN   NaN
2452           PR    total  1993         NaN   NaN

In [25]:
#lets see whcih region has null state
mergedPopAbbre.loc[mergedPopAbbre['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [27]:
#it means 'PR and USA ' state entries are nulll which is not present in abbrevation
#we have to add it mannualy
mergedPopAbbre.loc[mergedPopAbbre['state/region'] == 'PR', 'state'] = 'Puerto Rico'
mergedPopAbbre.loc[mergedPopAbbre['state/region'] == 'USA', 'state'] = 'United States'

mergedPopAbbre.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [28]:
#Step 3 : merge the result with the area data
final = pd.merge(mergedPopAbbre, areas, on = 'state', how = 'left')
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0

In [30]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [35]:
#there are null values in the area
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [37]:
#we will drop the united state because it is not relevent in current queiry
final.dropna(inplace=True)
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0

In [38]:
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [43]:
#now we are set to perform query

#filter the 2010 data
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

state/region   ages  year  population       state  area (sq. mi)
3             AL  total  2010   4785570.0     Alabama        52423.0
91            AK  total  2010    713868.0      Alaska       656425.0
101           AZ  total  2010   6408790.0     Arizona       114006.0
189           AR  total  2010   2922280.0    Arkansas        53182.0
197           CA  total  2010  37333601.0  California       163707.0

In [44]:
#compute population density 

#firstly re index the data
data2010.set_index('state', inplace=True)
data2010.head()

state/region   ages  year  population  area (sq. mi)
state                                                          
Alabama              AL  total  2010   4785570.0        52423.0
Alaska               AK  total  2010    713868.0       656425.0
Arizona              AZ  total  2010   6408790.0       114006.0
Arkansas             AR  total  2010   2922280.0        53182.0
California           CA  total  2010  37333601.0       163707.0

In [47]:
density = data2010['population']/data2010['area (sq. mi)']
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [50]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [51]:
#check end of list
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64